In [1]:
import csv
from datetime import datetime
import time
import random
import pickle

import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.stats import weightedtau, kendalltau
from scipy.stats import norm
from scipy.linalg import null_space

import matplotlib.pyplot as plt

from pprint import pprint
from copy import deepcopy

%pip install trueskill
import trueskill

  Preparing metadata (setup.py) ... done
  Created wheel for trueskill: filename=trueskill-0.4.5-py3-none-any.whl size=18049 sha256=792061ee8a5948eecbf69078074e80ff1eb275c837d40b137f904bf890f86c5e
  Stored in directory: /root/.cache/pip/wheels/b9/4f/29/c79f0a2956775524c7a23638ac2b6fbb516c680f8e5eed9b53
Successfully built trueskill


# First, parse game data

In [3]:
dt_str = '%d %B %Y %H:%M:%S'

dt_lim = datetime.strptime('06 August 2004 18:13:50', dt_str) #first game in HeadToHead

players = set()

# each entry is a tuple ([list of players], [list of scores])
matches = []

# needed for iteration
cur_game = -1
cur_players = []
cur_scores = []

# also, filter out all games where every player has no score!
with open('FreeForAll.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        date = datetime.strptime(row[0], dt_str)
        score = int(row[6])
        if date < dt_lim:
            game = int(row[1])
            player = row[4]
            score = int(row[6])

            # next, decide if this row is from the same match
            # as the last row, or a different match
            if game == cur_game:
                cur_players.append(player)
                cur_scores.append(score)
            else:
                if cur_game > 0 and np.sum(np.abs(cur_scores)):
                    # append cur_players, cur_scores to matches
                    matches.append((cur_players, cur_scores))
                    # add cur_players to players
                    players.update(cur_players)

                # reset cur_game, cur_players, cur_scores
                cur_game = game
                cur_players = [player]
                cur_scores = [score]
        else:
            break

players=list(players) # list of players

# Helper function for computing PageRank rankings

In [4]:
##################################################
# COMPUTE PAGERANK
##################################################

# given probability transition matrix P
# where P_{v,w} = Prob(w -> v)
# find pagerank scores with restart probability r
def compute_pr(P, r, n, eps=1e-8):
    x = np.ones(n) / n*1.0
    flag = True
    t=0
    while flag:
        x_new = (1-r)*P*x
        x_new = x_new + np.ones(n) * r / n
        diff = np.linalg.norm(x_new - x)
        if np.linalg.norm(x_new - x,ord=1) < eps and t > 100:
            flag = False
        t=t+1
        x = x_new
    return x

# Create hypergraph rankings

Rankings are a |V| x 1 vector, where the v-th entry is the PageRank score (or TrueSkill rating)  of vertex v

In [ ]:
pi_list = matches
universe = np.array(list(players))
# first create these matrices
# R = |E| x |V|, R(e, v) = lambda_e(v)
# W = |V| x |E|, W(v, e) = w(e) 1(v in e)

m = len(pi_list) # number of hyperedges
n = len(universe) # number of items to be ranked
R = np.zeros([m, n])
W = np.zeros([n, m])

for i in range(len(pi_list)):
    pi, scores = pi_list[i]
    if len(pi) > 1:
        for j in range(len(pi)):
            v = pi[j]
            v = np.where(universe == v)[0][0] #equivalent to universe.index(v) but for np arrays
            R[i, v] = np.exp(scores[j])
            W[v,i] = (np.std(scores) + 1.0)
        R[i, :] = R[i,:] / sum(R[i,:])

# first, normalize W
Wnorm=W/W.sum(axis=1)[:,None]
Ws = sparse.csr_matrix(Wnorm)
Rs = sparse.csr_matrix(R)

# create prob trans matrices
P = np.transpose(Ws.dot(Rs))

# create rankings
r=0.40
rankings_hg = compute_pr(P, r, n, eps=1e-8).flatten()

# Create G^H, MC3 rankings

## G^H rankings

In [ ]:
# Create matrix A, where A_{u,v} is given in Eq 10
def compute_gh_weights(R, W, P):
    E, V = R.shape
    A = np.zeros([V,V]) # to return

    # first, create edge weight vector
    WE = np.zeros(E)
    # for each edge, find first non-zero value that is >0
    for e in range(E):
        WE[e] = W[np.where(W[:,e] > 0)[0],e][0]

    # iterate over edges, add w(e) * gam_e(u) * gam_e(v) term
    # for each pair of vertices u,v \in e
    for e in range(E):
        nodes_in_e = np.nonzero(R[e,:])[0]
        for u in nodes_in_e:
            for v in nodes_in_e:
                A[u,v] += WE[e] * R[e,u] * R[e,v]
    return A

# create A, then find pagerank scores of random walk on A

# get probability transition matrix
A=compute_gh_weights(R, W, P)
P = A/A.sum(axis=1)[:,None]
P=P.T
P = sparse.csr_matrix(P)

# compute pagerank scores
r=0.40
rankings_gh = compute_pr(P, r, n, eps=1e-8).flatten()

## MC3 rankings

In [ ]:
n = len(universe)

Pd = np.zeros([n, n]) # d for dwork

for i in universe:
    i_counts = np.zeros(n) # number of ways i can go to any other vertex
    i_deg = 0 #number of hyperedges where i can traverse to some other vertex

    i_index = np.where(universe == i)
    for pi, scores in pi_list:
        if i in pi and len(pi) > 1:
            pi_filtered = pi[pi.index(i)+1:] #everything ranked better than i

            # if i can use this hyperedge
            if len(pi_filtered) > 0:
                # essentially, for each j in pi_filtered
                # grab k=universe.index(j) and increment i_counts[k] by 1/len(pi)
                i_counts[np.where(np.isin(universe, pi_filtered))] += 1/len(pi)

            i_counts[i_index] += 1 - (len(pi_filtered) / len(pi))
            i_deg += 1
    if i_deg > 0:
        i_counts /= i_deg
    else:
        i_counts[i_index] = 1
    Pd[i_index,:] = i_counts

Pd = np.transpose(Pd) # since we're using column vectors
Pd = sparse.csr_matrix(Pd)

# create MC3 rankings
r=0.40

rankings_mc3 = compute_pr(Pd, r, n, eps=1e-8).flatten()

# Create TrueSkill rankings

In [ ]:
# simulate the change in TrueSkill ratings when a Free-For-All match is played
def play_match(match, ts_ranking):
    p, s = match
    cur_ranks = []
    for player in p:
        if player in ts_ranking:
            cur_ranks.append([ts_ranking[player]])
        else:
            cur_ranks.append([trueskill.Rating()])
    # lower rank = better player for trueskill.rate function, so we turn scores into -1*scores
    match_res = trueskill.rate(cur_ranks, ranks=[-1*i for i in s])
    for i in range(len(p)):
        player = p[i]
        ts_ranking[player] = match_res[i][0]

In [ ]:
trueskill_rankings={} # dict mapping player -> TrueSkill rating object

# simulate all matches being played, in order
for match in matches:
    play_match(match, trueskill_rankings)

rankings_ts = [trueskill_rankings[player].mu for player in players] # deterministic TrueSkill ratings list

# Evaluate 4 rankings

In [ ]:
# Evaluating a 1v1 game with a deterministic ranking of players
# INPUTS:
# game_players: list of players in the match
# game_scores: list of scores in the match (corresponding to game_players)
# all_players: list of all players in all matches
# ranks: one of the 4 rankings computed above

# OUTPUT:
# can_eval: False if game ends in tie, True otherwise (we ignore tie games)
# res: 1 if ranks correctly predicts the match, 0 if not
def eval_game_h2h(game_players, game_scores, all_players, ranks):
    players_ranked_prev = [player for player in game_players if player in all_players]
    if len(players_ranked_prev) == 2:
        # get scores for players previously ranked
        scores_prev = [game_scores[game_players.index(player)] for player in players_ranked_prev]
        ranks_prev = [ranks[all_players.index(player)] for player in players_ranked_prev]

        # make sure there isn't a tie
        if scores_prev[0] != scores_prev[1]:
            can_eval = True

            # check if ranked correctly
            if sum(np.argsort(scores_prev) == np.argsort(ranks_prev)) == 2:
                res = True
            else:
                res = False
        else:
            can_eval = False
            res = False
    else:
        can_eval = False
        res = False
    return (can_eval, int(res))

# Evaluating a 1v1 game with TS probabilistic procedure. Same inputs/outputs as above.
def eval_game_h2h_trueskill(game_players, game_scores, all_players, ts_ranking):
    players_ranked_prev = [player for player in game_players if player in all_players]
    if len(players_ranked_prev) == 2:
        # get scores for players previously ranked
        scores_prev = [game_scores[game_players.index(player)] for player in players_ranked_prev]
        ts_ranks_prev = [ts_ranking[player] for player in players_ranked_prev]

        # make sure there isn't a tie
        if scores_prev[0] != scores_prev[1]:
            can_eval = True

            # compare rating distributions between two players
            # (for simplicity, do not consider draw probability)
            mu0, sigma0 = ts_ranks_prev[0]
            mu1, sigma1 = ts_ranks_prev[1]
            p = 1 - norm.cdf(-1.0 * (mu0 - mu1) / (sigma0**2 + sigma1**2))
            if (p > 0.5 and scores_prev[0] > scores_prev[1]) or (p < 0.5 and scores_prev[0] < scores_prev[1]):
                res = True
            else:
                res = False

        else:
            can_eval = False
            res = False
    else:
        can_eval = False
        res = False
    return (can_eval, res)

Go through each game and use each of the different rankings to predict the winner. Compare to actual winner.

In [ ]:
cur_game = -1
cur_players = []
cur_scores = []

results_hg=[]
results_gh=[]
results_mc3=[]
results_ts=[]
results_ts_prob=[]

with open('HeadToHead.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        game = int(row[1])
        player = row[4]
        score = int(row[6])

        # next, decide if this row is from the same match
        # as the last row, or a different match
        if game == cur_game:
            cur_players.append(player)
            cur_scores.append(score)
        else:
            if cur_game > 0 and np.sum(np.abs(cur_scores)) > 0:
                # evaluate game
                can_eval, hg_match_res = eval_game_h2h(cur_players, cur_scores, players, rankings_hg)
                can_eval, gh_match_res = eval_game_h2h(cur_players, cur_scores, players, rankings_gh)
                can_eval, mc3_match_res = eval_game_h2h(cur_players, cur_scores, players, rankings_mc3)
                can_eval, ts_match_res = eval_game_h2h(cur_players, cur_scores, players, rankings_ts)

                can_eval, ts_prob_match_res = eval_game_h2h_trueskill(cur_players, cur_scores, players, trueskill_rankings)

                if can_eval:
                    results_hg.append(hg_match_res)
                    results_gh.append(gh_match_res)
                    results_mc3.append(mc3_match_res)
                    results_ts.append(ts_match_res)

                    results_ts_prob.append(ts_prob_match_res)

            # reset cur_game, cur_players, cur_scores
            cur_game = game
            cur_players = [player]
            cur_scores = [score]

In [ ]:
num_games = len(results_hg)
print('Hypergraph accuracy: {}'.format(sum(results_hg) * 1.0 / num_games))
print('Clique Graph accuracy: {}'.format(sum(results_gh) * 1.0 / num_games))
print('Dwork MC3 accuracy: {}'.format(sum(results_mc3) * 1.0 / num_games))
print('TrueSkill accuracy: {}'.format(sum(results_ts) * 1.0 / num_games))
print('TrueSkill accuracy, probabilistic decision procedure: {}'.format(sum(results_ts_prob) * 1.0 / num_games))

Hypergraph accuracy: 0.7113685450618495
Clique Graph accuracy: 0.6112311015118791
Dwork MC3 accuracy: 0.5293540153151384
TrueSkill accuracy: 0.7345376006283134
TrueSkill accuracy, probabilistic decision procedure: 0.7345376006283134


# Differences between hypergraph and TS rankings

We compute the percentage of matches which HG/TS differ on

In [ ]:
hg_only=0
ts_only=0
both=0
for i in range(num_games):
    if results_hg[i] > 0 and results_ts[i] > 0:
        both += 1
    elif results_hg[i] > 0:
        hg_only += 1
    elif results_ts[i] > 0:
        ts_only += 1

In [ ]:
print('% of Matches predicted correctly by both TrueSkill and hypergraph: {}'.format(both / num_games))
print('% of Matches predicted correctly by only TrueSkill: {}'.format(ts_only / num_games))
print('% of Matches predicted correctly by only hypergraph: {}'.format(hg_only / num_games))

% of Matches predicted correctly by both TrueSkill and hypergraph: 0.6220302375809935
% of Matches predicted correctly by only TrueSkill: 0.11250736304731986
% of Matches predicted correctly by only hypergraph: 0.08933830748085608
